In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
import warnings
from naming_scheme import type_and_suffix
from collections import deque
import cox,binomial,conditional_lr,icd_data_process
from MAFLD_tag import psm_result
from MAFLD_tag import NFS_tag
from MAFLD_tag import get_MAFLD


In [2]:
!pipreqs .

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in ./requirements.txt


# Generate label

In [ ]:
get_MAFLD('data_original')

# Run PSM.R to get PSM result 

In [ ]:
psm_result()
NFS_tag()

# Section 1

In [ ]:
task=type_and_suffix('disease','all',)

cox.for_cox(task)

cox.cox_regression(task)

binomial.disease_pairs(task)

conditional_lr.cal_lr(task,n_neighbors=5,conditional=False)


In [ ]:

tasks=type_and_suffix('disease','all',)

tags=[0,1]
for subtask in tasks.subtasks:
    for tag in tags:
        task=type_and_suffix('disease',subtask,tag)

        cox.cox_regression(task)
        
        binomial.disease_pairs_subgroup(task)
        
        conditional_lr.cal_lr(task,n_neighbors=5,conditional=False)

cox.create_table_subgroup(tasks)        

conditional_lr.create_table_subgroup(tasks)

In [ ]:
task=type_and_suffix('disease','all',)
cox.for_cox_sens(task)
cox.cox_regression(task,sens=True)
cox.create_table_sens(task)

In [ ]:
task=type_and_suffix('death','all',)
cox.cox_regression(task,target_disease='NFS_above',custom='_NFS_above')
cox.cox_regression(task,target_disease='NFS_below',custom='_NFS_below')
cox.create_table_nfs(task)

# Section 2

In [ ]:
task=type_and_suffix('death','all',)

cox.cox_regression(task)

In [ ]:
tags=[0,1]
for subtask in task.subtasks:
    for tag in tags:
        task=type_and_suffix('death',subtask,tag)
        cox.cox_regression(task)
task=type_and_suffix('death','all',)
cox.create_table_subgroup(task)

In [ ]:
task=type_and_suffix('death','all',)
cox.cox_regression(task,sens=True)
cox.create_table_sens(task)

# Section 3

In [ ]:
Deaths=pd.read_csv('cox/cox_result_death_all.csv',index_col=0)
for Death in Deaths.index:
    frequency=int(Deaths.loc[Death,'frequency']*0.05)
    task=type_and_suffix('death','all',Death_cause=Death)
    cox.death_cox_regression(task)
    binomial.disease_death_pairs(task,frequency)
    conditional_lr.cal_lr(task,n_neighbors=5,conditional=False)
    conditional_lr.cal_lr_death(task,n_neighbors=5)

In [ ]:
Death_data=pd.DataFrame()
Deaths=pd.read_csv('cox/cox_result_death_all.csv',index_col=0)
for Death in Deaths.index:
    data=pd.read_csv('con_lr/T2_death_'+Death+'.csv')
    data.insert(1,'Death_cause',Death)
    Death_data=pd.concat([Death_data,data])
Death_data.to_csv('con_lr_subgroup/lr_death_merge.csv',index=False)

In [ ]:
Death_data=pd.DataFrame()
Deaths=pd.read_csv('cox/cox_result_death_all.csv',index_col=0)
for Death in Deaths.index:
    data=pd.read_csv('cox/cox_result_death_'+Death+'.csv')
    data.insert(1,'Death_cause',Death)
    Death_data=pd.concat([Death_data,data])
Death_data.to_csv('cox_subgroup/cox_death_merge.csv',index=False)

# Data used to draw trajectory

In [ ]:
data=pd.read_csv('con_lr/T2_disease_all.csv',index_col=0)
data.to_csv('Draw/Disease_original_all.csv')
conditional_lr.paint_edge(data).to_csv('Draw/Disease_simplified_all.csv')